In [1]:
import pandas as pd
import numpy as np
import math
import json
import os
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='solarizedd')
plt.rcParams['figure.figsize'] = (20.0, 10.0)

import data_utils_mt.utils as utils
import seaborn as sns
from sklearn.pipeline import Pipeline
import datetime as dt
from xgboost import XGBClassifier

ROOT_DIR = '../..'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
DATA_RAW = os.path.join(DATA_DIR, 'raw')
DATA_INTERIM = os.path.join(DATA_DIR, 'interim')
DATA_EXTERNAL = os.path.join(DATA_DIR, 'external')
DATA_PROCESSED = os.path.join(DATA_DIR, 'processed')
SRC = os.path.join(ROOT_DIR, 'src')

STATIC_DATASET_PATH = os.path.join(DATA_PROCESSED, 'static_cluster3d.pkl')

import sys
sys.path.append(SRC)

import src.data.preprocessing as pp
import src.data.success_dataset as sd
import src.data.missing_data as md
import src.evaluation.offer_success as evos
from sklearn.cluster import KMeans

In [2]:
data = pd.read_pickle(STATIC_DATASET_PATH)
print(data.shape)
data.head()

(76277, 39)


,person,time,offer_id,age,became_member_on,gender,income,missing_demographics,member_epoch_days,difficulty,...,ward_12,dbscan_10,3d_kmeans_3,3d_ward_3,3d_ward_9,3d_ward_19,3d_gmm_3,3d_gmm_16,3d_dbscan_02_20,3d_dbscan_05_100
0,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,33.0,2017-04-21,M,72000.0,0,17277,0.0,...,8.0,1.0,0.0,1.0,3.0,4.0,1.0,13.0,-1.0,0.0
1,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed,33.0,2017-04-21,M,72000.0,0,17277,0.0,...,8.0,1.0,0.0,1.0,3.0,4.0,1.0,13.0,-1.0,0.0
2,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d,33.0,2017-04-21,M,72000.0,0,17277,5.0,...,8.0,1.0,0.0,1.0,3.0,4.0,1.0,13.0,-1.0,0.0
3,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4,33.0,2017-04-21,M,72000.0,0,17277,10.0,...,8.0,1.0,0.0,1.0,3.0,4.0,1.0,13.0,-1.0,0.0
4,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5,33.0,2017-04-21,M,72000.0,0,17277,10.0,...,8.0,1.0,0.0,1.0,3.0,4.0,1.0,13.0,-1.0,0.0


In [3]:
data.columns

Index(['person', 'time', 'offer_id', 'age', 'became_member_on', 'gender',
       'income', 'missing_demographics', 'member_epoch_days', 'difficulty',
       'duration', 'offer_type', 'reward_t', 'channel_social', 'channel_email',
       'channel_web', 'channel_mobile', 'completed', 'expected_finish',
       'finish', 'success', 'view_time', 'viewed', 'actual_reward',
       'profit_in_duration', 'profit_until_complete', 'spent_in_duration',
       'spent_until_complete', 'kmeans_8', 'ward_12', 'dbscan_10',
       '3d_kmeans_3', '3d_ward_3', '3d_ward_9', '3d_ward_19', '3d_gmm_3',
       '3d_gmm_16', '3d_dbscan_02_20', '3d_dbscan_05_100'],
      dtype='object')

### Lagged success (how many times each offer was shown, and how many of them were a success?)

In [4]:
portfolio = pd.read_json(os.path.join(DATA_RAW, 'portfolio.json'), orient='records', lines=True)
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [5]:
def fill_one_lagged_success(user_data, current_time, portfolio):
    """ 
    For a given time, and a given user, it counts how many times each offer was shown, 
    and how many of those were a success (the rate of success could be easily calculated 
    afterwards).
    offer_id_n: keeps track of how many times the offer was shown
    offer_id_success: keeps track of how many times the offer was successful
    """
    feat_names = ['offer_type', 'duration', 'difficulty', 'reward']
    ohe_feats = ['channel_web', 'channel_email', 'channel_social', 'channel_mobile']
    
    # Some type conversion (data may have NaNs and converts to float)
    portfolio_t = portfolio.copy()
    portfolio_t[['difficulty', 'duration', 'reward']] = portfolio_t[
        ['difficulty', 'duration', 'reward']].astype(float)
    
    # Create the results containers
    feats = portfolio_t.id.tolist()
    for feat_name in feat_names:
        feats += [feat_name + '_{}'.format(d) 
                  for d in portfolio_t[feat_name].unique().tolist()]
    label_feats = np.setdiff1d(feat_names, ['reward']).tolist()
    feats += ohe_feats
    
    shown = {'{}_n'.format(offer): 0 for offer in feats}
    success = {'{}_success'.format(offer): 0 for offer in feats}
    res = {**shown, **success}
    
    old_offers =  user_data[user_data.time < current_time]
    for i, row in old_offers.iterrows():
        res['{}_n'.format(row.offer_id)] += 1
        for feat_name in label_feats:
            res[feat_name + '_{}_n'.format(row[feat_name])] += 1
        res['reward_{}_n'.format(row['reward_t'])] += 1
        for feat_name in ohe_feats:
            if row[feat_name] == 1:
                res['{}_n'.format(feat_name)] += 1
        
        if row.success == 1:
            res['{}_success'.format(row.offer_id)] += 1  
            for feat_name in label_feats:
                res[feat_name + '_{}_success'.format(row[feat_name])] += 1
            res['reward_{}_success'.format(row['reward_t'])] += 1
            for feat_name in ohe_feats:
                if row[feat_name] == 1:
                    res['{}_success'.format(feat_name)] += 1
            
    return pd.Series(res)

In [6]:
user = data[data.person == data.person[7]]
user[['time', 'offer_id', 'success']]

,time,offer_id,success
7,0,3f207df678b143eea3cee63160fa8bed,0
8,168,2298d6c36e964ae4a3e7e9706d1fb8c2,1
9,336,5a8bc65990b245e5a138643cd4eb9837,0
10,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,1
11,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,1


In [7]:
fill_one_lagged_success(user, 576, portfolio)

ae264e3637204a6fb9bb56bc8210ddfd_n          0
4d5c57ea9a6940dd891ad53e9dbe8da0_n          0
3f207df678b143eea3cee63160fa8bed_n          1
9b98b8c7a33c4b65b9aebfe6a799e6d9_n          1
0b1e1539f2cc45b7b9fa7c272da2e1d7_n          1
2298d6c36e964ae4a3e7e9706d1fb8c2_n          1
fafdcd668e3743c1bb461111dcafc2a4_n          0
5a8bc65990b245e5a138643cd4eb9837_n          1
f19421c1d4aa40978ebb69ca19b0e20d_n          0
2906b810c7d4411798c6938adc9daaa5_n          0
offer_type_bogo_n                           1
offer_type_informational_n                  2
offer_type_discount_n                       2
duration_7.0_n                              2
duration_5.0_n                              0
duration_4.0_n                              1
duration_10.0_n                             1
duration_3.0_n                              1
difficulty_10.0_n                           0
difficulty_0.0_n                            2
difficulty_5.0_n                            1
difficulty_20.0_n                 

In [8]:
def fill_user_lagged_success(user_data, portfolio):
    """ Fills the lagged success for all the records in one customer. """
    return user_data.join(user_data.apply(
        lambda x: fill_one_lagged_success(user_data, x.time, portfolio), axis=1))

In [9]:
user2 = fill_user_lagged_success(user, portfolio)

In [10]:
user2

,person,time,offer_id,age,became_member_on,gender,income,missing_demographics,member_epoch_days,difficulty,...,difficulty_7.0_success,reward_10.0_success,reward_0.0_success,reward_5.0_success,reward_3.0_success,reward_2.0_success,channel_web_success,channel_email_success,channel_social_success,channel_mobile_success
7,0011e0d4e6b944f998e987f904e8c1e5,0,3f207df678b143eea3cee63160fa8bed,40.0,2018-01-09,O,57000.0,0,17540,0.0,...,0,0,0,0,0,0,0,0,0,0
8,0011e0d4e6b944f998e987f904e8c1e5,168,2298d6c36e964ae4a3e7e9706d1fb8c2,40.0,2018-01-09,O,57000.0,0,17540,7.0,...,0,0,0,0,0,0,0,0,0,0
9,0011e0d4e6b944f998e987f904e8c1e5,336,5a8bc65990b245e5a138643cd4eb9837,40.0,2018-01-09,O,57000.0,0,17540,0.0,...,1,0,0,0,1,0,1,1,1,1
10,0011e0d4e6b944f998e987f904e8c1e5,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,40.0,2018-01-09,O,57000.0,0,17540,20.0,...,1,0,0,0,1,0,1,1,1,1
11,0011e0d4e6b944f998e987f904e8c1e5,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,40.0,2018-01-09,O,57000.0,0,17540,5.0,...,1,0,0,1,1,0,2,2,1,1


In [11]:
def fill_lagged_success(data, portfolio):
    """ Fills the lagged success for all the dataset. """
    return data.groupby('person').apply(
        lambda x: fill_user_lagged_success(x, portfolio))

In [12]:
%time data2 = fill_lagged_success(data, portfolio)

CPU times: user 17min 57s, sys: 4.5 s, total: 18min 2s
Wall time: 20min 30s


In [13]:
data2.head()

,person,time,offer_id,age,became_member_on,gender,income,missing_demographics,member_epoch_days,difficulty,...,difficulty_7.0_success,reward_10.0_success,reward_0.0_success,reward_5.0_success,reward_3.0_success,reward_2.0_success,channel_web_success,channel_email_success,channel_social_success,channel_mobile_success
0,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,33.0,2017-04-21,M,72000.0,0,17277,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed,33.0,2017-04-21,M,72000.0,0,17277,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d,33.0,2017-04-21,M,72000.0,0,17277,5.0,...,0,0,0,0,0,0,0,0,0,0
3,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4,33.0,2017-04-21,M,72000.0,0,17277,10.0,...,0,0,0,0,0,0,0,0,0,0
4,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5,33.0,2017-04-21,M,72000.0,0,17277,10.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data2['2298d6c36e964ae4a3e7e9706d1fb8c2_success'].value_counts()

0    68611
1     7106
2      544
3       16
Name: 2298d6c36e964ae4a3e7e9706d1fb8c2_success, dtype: int64

In [15]:
data2.filter(regex='.*_n', axis=1).head()

,ae264e3637204a6fb9bb56bc8210ddfd_n,4d5c57ea9a6940dd891ad53e9dbe8da0_n,3f207df678b143eea3cee63160fa8bed_n,9b98b8c7a33c4b65b9aebfe6a799e6d9_n,0b1e1539f2cc45b7b9fa7c272da2e1d7_n,2298d6c36e964ae4a3e7e9706d1fb8c2_n,fafdcd668e3743c1bb461111dcafc2a4_n,5a8bc65990b245e5a138643cd4eb9837_n,f19421c1d4aa40978ebb69ca19b0e20d_n,2906b810c7d4411798c6938adc9daaa5_n,...,difficulty_7.0_n,reward_10.0_n,reward_0.0_n,reward_5.0_n,reward_3.0_n,reward_2.0_n,channel_web_n,channel_email_n,channel_social_n,channel_mobile_n
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,1,1
2,0,0,1,0,0,0,0,1,0,0,...,0,0,2,0,0,0,1,2,1,2
3,0,0,1,0,0,0,0,1,1,0,...,0,0,2,1,0,0,2,3,2,3
4,0,0,1,0,0,0,1,1,1,0,...,0,0,2,1,0,1,3,4,3,4


In [16]:
data2.filter(regex='.*_success', axis=1).head()

,ae264e3637204a6fb9bb56bc8210ddfd_success,4d5c57ea9a6940dd891ad53e9dbe8da0_success,3f207df678b143eea3cee63160fa8bed_success,9b98b8c7a33c4b65b9aebfe6a799e6d9_success,0b1e1539f2cc45b7b9fa7c272da2e1d7_success,2298d6c36e964ae4a3e7e9706d1fb8c2_success,fafdcd668e3743c1bb461111dcafc2a4_success,5a8bc65990b245e5a138643cd4eb9837_success,f19421c1d4aa40978ebb69ca19b0e20d_success,2906b810c7d4411798c6938adc9daaa5_success,...,difficulty_7.0_success,reward_10.0_success,reward_0.0_success,reward_5.0_success,reward_3.0_success,reward_2.0_success,channel_web_success,channel_email_success,channel_social_success,channel_mobile_success
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
success_cols = data2.columns.str.extract('(.*)_success').dropna().values.flatten().tolist()
success_cols

['ae264e3637204a6fb9bb56bc8210ddfd',
 '4d5c57ea9a6940dd891ad53e9dbe8da0',
 '3f207df678b143eea3cee63160fa8bed',
 '9b98b8c7a33c4b65b9aebfe6a799e6d9',
 '0b1e1539f2cc45b7b9fa7c272da2e1d7',
 '2298d6c36e964ae4a3e7e9706d1fb8c2',
 'fafdcd668e3743c1bb461111dcafc2a4',
 '5a8bc65990b245e5a138643cd4eb9837',
 'f19421c1d4aa40978ebb69ca19b0e20d',
 '2906b810c7d4411798c6938adc9daaa5',
 'offer_type_bogo',
 'offer_type_informational',
 'offer_type_discount',
 'duration_7.0',
 'duration_5.0',
 'duration_4.0',
 'duration_10.0',
 'duration_3.0',
 'difficulty_10.0',
 'difficulty_0.0',
 'difficulty_5.0',
 'difficulty_20.0',
 'difficulty_7.0',
 'reward_10.0',
 'reward_0.0',
 'reward_5.0',
 'reward_3.0',
 'reward_2.0',
 'channel_web',
 'channel_email',
 'channel_social',
 'channel_mobile']

In [18]:
for col in success_cols:
    data2['{}_success_ratio'.format(col)] = data2['{}_success'.format(col)] / (data2['{}_n'.format(col)] + 1e-5)

In [19]:
print(data2.shape)
data2.head()

(76277, 135)


,person,time,offer_id,age,became_member_on,gender,income,missing_demographics,member_epoch_days,difficulty,...,difficulty_7.0_success_ratio,reward_10.0_success_ratio,reward_0.0_success_ratio,reward_5.0_success_ratio,reward_3.0_success_ratio,reward_2.0_success_ratio,channel_web_success_ratio,channel_email_success_ratio,channel_social_success_ratio,channel_mobile_success_ratio
0,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,33.0,2017-04-21,M,72000.0,0,17277,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed,33.0,2017-04-21,M,72000.0,0,17277,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d,33.0,2017-04-21,M,72000.0,0,17277,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4,33.0,2017-04-21,M,72000.0,0,17277,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5,33.0,2017-04-21,M,72000.0,0,17277,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
data2.filter(regex='.*_ratio').describe()

,ae264e3637204a6fb9bb56bc8210ddfd_success_ratio,4d5c57ea9a6940dd891ad53e9dbe8da0_success_ratio,3f207df678b143eea3cee63160fa8bed_success_ratio,9b98b8c7a33c4b65b9aebfe6a799e6d9_success_ratio,0b1e1539f2cc45b7b9fa7c272da2e1d7_success_ratio,2298d6c36e964ae4a3e7e9706d1fb8c2_success_ratio,fafdcd668e3743c1bb461111dcafc2a4_success_ratio,5a8bc65990b245e5a138643cd4eb9837_success_ratio,f19421c1d4aa40978ebb69ca19b0e20d_success_ratio,2906b810c7d4411798c6938adc9daaa5_success_ratio,...,difficulty_7.0_success_ratio,reward_10.0_success_ratio,reward_0.0_success_ratio,reward_5.0_success_ratio,reward_3.0_success_ratio,reward_2.0_success_ratio,channel_web_success_ratio,channel_email_success_ratio,channel_social_success_ratio,channel_mobile_success_ratio
count,76277.000000,76277.000000,76277.0,76277.000000,76277.000000,76277.000000,76277.000000,76277.0,76277.000000,76277.000000,...,76277.000000,76277.000000,76277.0,76277.000000,76277.000000,76277.000000,76277.000000,76277.000000,76277.000000,76277.000000
mean,0.060752,0.061554,0.0,0.052214,0.049980,0.097540,0.109875,0.0,0.078607,0.053941,...,0.097540,0.110455,0.0,0.147644,0.097540,0.148621,0.267862,0.257108,0.256908,0.253037
std,0.236452,0.238293,0.0,0.219411,0.215179,0.294205,0.310737,0.0,0.266397,0.222797,...,0.294205,0.306856,0.0,0.339739,0.294205,0.347324,0.382153,0.357700,0.394048,0.362347
min,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.499998,0.499998,0.499998,0.499998
max,0.999998,0.999997,0.0,0.999997,0.999997,0.999997,0.999997,0.0,0.999997,0.999997,...,0.999997,0.999998,0.0,0.999998,0.999997,0.999998,0.999998,0.999998,0.999998,0.999998


In [21]:
data2.to_pickle(os.path.join(DATA_PROCESSED, 'static_cluster_lagged.pkl'))